In [ ]:
# This script reads your two CSVs, extracts **hard constraints (needs)** and **real-world trade-offs**,
# and prepares a joined "bundles" table you can use for deterministic optimization.
#
# It will:
# 1) Load `mac_lineup_current_2025-08-19.csv` and `ipad_lineup_current_2025-08-19.csv` from /mnt/data
#    (If not found, it will fall back to the sample data you shared.)
# 2) Extract "needs" columns (constraints) and "trade-off" columns into two tidy tables.
# 3) Create simple 1-device bundles (Mac only) and 2-device bundles (Mac + iPad).
# 4) Save results to CSVs and display them as interactive tables.
#
# You can tweak NEEDS_THRESHOLDS and which columns are considered "needs" vs "trade-offs" below.

import os
import io
import json
import pandas as pd

In [5]:

PARENT_DIR = "apple_data"
MAC_PATH = f"{PARENT_DIR}/mac_lineup_current_2025-08-19.csv"
IPAD_PATH = f"{PARENT_DIR}/ipad_lineup_current_2025-08-19.csv"

sample_mac = """name,category,url,chip,ram_gb,storage_gb,storage_tb,battery_hours,weight_kg,price_inr,ports,display_inches,notes,learning_hours,maintenance_hours_per_year,power_adequacy_score
MacBook Air 13″ (M4),macbook,https://www.apple.com/in/shop/buy-mac/macbook-air,M4,16,256,2,15.0,1.24,99900.0,"[""MagSafe 3"", ""2x Thunderbolt 4 (USB-C)"", ""3.5mm""]",13.6,Up to 18h video; supports up to two external displays.,1.0,4,7.5
MacBook Air 15″ (M4),macbook,https://www.apple.com/in/shop/buy-mac/macbook-air,M4,16,256,2,15.0,1.51,124900.0,"[""MagSafe 3"", ""2x Thunderbolt 4 (USB-C)"", ""3.5mm""]",15.3,Up to 18h video; supports up to two external displays.,1.0,4,7.5
"""

sample_ipad = """name,category,url,chip,ram_gb,storage_gb,storage_tb,battery_hours,weight_kg,price_inr,ports,display_inches,notes,learning_hours,maintenance_hours_per_year,power_adequacy_score
iPad Pro 11″ (M4),ipad,https://www.apple.com/in/ipad-pro/,M4,,256,2.0,10.0,0.444,99900.0,USB‑C (Thunderbolt/USB 4),11.1,Wi‑Fi model weight; 'From' price (Wi‑Fi) in India,,,
iPad Pro 13″ (M4),ipad,https://www.apple.com/in/ipad-pro/,M4,,256,2.0,10.0,0.579,129900.0,USB‑C (Thunderbolt/USB 4),13.0,Wi‑Fi model weight; 'From' price (Wi‑Fi) in India,,,
"""

def load_csv_or_sample(path, sample_text):
    #if os.path.exists(path):
    return pd.read_csv(path)
    #else:
    #    return pd.read_csv(io.StringIO(sample_text))

mac_df = load_csv_or_sample(MAC_PATH, sample_mac)
ipad_df = load_csv_or_sample(IPAD_PATH, sample_ipad)

mac_df

,name,category,url,chip,ram_gb,storage_gb,storage_tb,battery_hours,weight_kg,price_inr,ports,display_inches,notes,learning_hours,maintenance_hours_per_year,power_adequacy_score
0,MacBook Air 13″ (M4),macbook,https://www.apple.com/in/shop/buy-mac/macbook-air,M4,16,256,2,15.0,1.24,99900.0,"[""MagSafe 3"", ""2x Thunderbolt 4 (USB-C)"", ""3.5...",13.6,Up to 18h video; supports up to two external d...,1.0,4,7.5
1,MacBook Air 15″ (M4),macbook,https://www.apple.com/in/shop/buy-mac/macbook-air,M4,16,256,2,15.0,1.51,124900.0,"[""MagSafe 3"", ""2x Thunderbolt 4 (USB-C)"", ""3.5...",15.3,Up to 18h video; supports up to two external d...,1.0,4,7.5
2,MacBook Pro 14″ (M4),macbook,https://www.apple.com/in/shop/buy-mac/macbook-pro,M4,16,512,2,16.0,1.55,169900.0,"[""MagSafe 3"", ""3x Thunderbolt 4 (USB-C)"", ""HDM...",14.2,Liquid Retina XDR.,1.5,5,8.3
3,MacBook Pro 14″ (M4 Pro or M4 Max),macbook,https://www.apple.com/in/shop/buy-mac/macbook-pro,M4 Pro or M4 Max,24,512,8,14.0,1.60,199900.0,"[""MagSafe 3"", ""3x Thunderbolt 5 (USB-C)"", ""HDM...",14.2,M4 Pro: up to 14h web; M4 Max: up to 13h web; ...,2.0,6,9.2
4,MacBook Pro 16″ (M4 Pro or M4 Max),macbook,https://www.apple.com/in/shop/buy-mac/macbook-pro,M4 Pro or M4 Max,24,512,8,17.0,2.14,249900.0,"[""MagSafe 3"", ""3x Thunderbolt 5 (USB-C)"", ""HDM...",16.2,M4 Pro: up to 17h web; M4 Max: up to 14h web; ...,2.0,6,9.6
5,"iMac (M4, two ports)",mac,https://www.apple.com/in/shop/buy-mac/imac,M4,16,256,2,NaN,NaN,134900.0,"[""2x Thunderbolt / USB 4""]",24.0,4.5K Retina; two-port configuration.,0.5,3,7.5
6,"iMac (M4, four ports)",mac,https://www.apple.com/in/shop/buy-mac/imac,M4,16,256,2,NaN,NaN,154900.0,"[""4x Thunderbolt 4"", ""Gigabit Ethernet""]",24.0,4.5K Retina; four-port configuration.,0.5,3,7.5
7,Mac mini (M4 or M4 Pro),mac,https://www.apple.com/in/shop/buy-mac/mac-mini,M4 or M4 Pro,16,256,2,NaN,0.67,59900.0,"[""Front: 2x USB-C, 3.5mm"", ""Back: 3x Thunderbo...",NaN,M4: up to 2TB storage; M4 Pro: up to 8TB; M4 P...,0.5,2,7.8
8,Mac Studio (M4 Max or M3 Ultra),mac,https://www.apple.com/in/shop/buy-mac/mac-studio,M4 Max or M3 Ultra,36,512,8,NaN,NaN,214900.0,"[""Front: 2x USB-C, SDXC"", ""Back: 4x Thunderbol...",NaN,M4 Max: up to 8TB; M3 Ultra: up to 16TB; base ...,1.0,3,9.8
9,Mac Pro (M2 Ultra),mac,https://www.apple.com/in/shop/buy-mac/mac-pro,M2 Ultra,64,1000,8,NaN,NaN,729900.0,[],NaN,Tower. Expansion via PCIe; ports vary by chassis.,1.0,6,9.2


In [3]:
ipad_df

,name,category,url,chip,ram_gb,storage_gb,storage_tb,battery_hours,weight_kg,price_inr,ports,display_inches,notes,learning_hours,maintenance_hours_per_year,power_adequacy_score
0,iPad Pro 11″ (M4),ipad,https://www.apple.com/in/ipad-pro/,M4,NaN,256,2.000,10.0,0.444,99900.0,USB‑C (Thunderbolt/USB 4),11.10,Wi‑Fi model weight; 'From' price (Wi‑Fi) in India,NaN,NaN,NaN
1,iPad Pro 13″ (M4),ipad,https://www.apple.com/in/ipad-pro/,M4,NaN,256,2.000,10.0,0.579,129900.0,USB‑C (Thunderbolt/USB 4),13.00,Wi‑Fi model weight; 'From' price (Wi‑Fi) in India,NaN,NaN,NaN
2,iPad Air 11″ (M3),ipad,https://www.apple.com/in/ipad-air/,M3,8.0,128,1.000,10.0,0.460,59900.0,USB‑C,10.86,Wi‑Fi model weight; 'From' price (Wi‑Fi) in India,NaN,NaN,NaN
3,iPad Air 13″ (M3),ipad,https://www.apple.com/in/ipad-air/,M3,8.0,128,1.000,10.0,0.616,79900.0,USB‑C,12.90,Wi‑Fi model weight; 'From' price (Wi‑Fi) in India,NaN,NaN,NaN
4,iPad 11″ (A16),ipad,https://www.apple.com/in/ipad-11/,A16,NaN,128,0.512,10.0,0.477,34900.0,USB‑C,10.86,Wi‑Fi model weight; 'From' price (Wi‑Fi) in India,NaN,NaN,NaN
5,iPad mini 8.3″ (A17 Pro),ipad,https://www.apple.com/in/ipad-mini/,A17 Pro,NaN,128,0.512,10.0,0.293,49900.0,USB‑C,8.30,Wi‑Fi model weight; 'From' price (Wi‑Fi) in India,NaN,NaN,NaN


In [4]:

# Clean basic types
for df in (mac_df, ipad_df):
    for col in ["ram_gb","storage_gb","storage_tb","battery_hours","weight_kg","price_inr","display_inches",
                "learning_hours","maintenance_hours_per_year","power_adequacy_score"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

# ---- 1) Define which columns map to needs vs trade-offs ----
NEEDS_COLS = [
    "chip", "ram_gb", "storage_tb", "battery_hours", "weight_kg", "display_inches", "ports"
]
TRADEOFF_COLS = [
    "price_inr", "learning_hours", "maintenance_hours_per_year", "power_adequacy_score"
]

# Thresholds for deterministic "needs" (edit to taste)
NEEDS_THRESHOLDS = {
    "ram_gb": 16,           # >=
    "storage_tb": 1.0,      # >=
    "battery_hours": 15.0,  # >=
    "weight_kg": 2.1,       # <= (handled specially below)
    # display_inches & ports are qualitative; we leave as-is for inspection
}

# ---- 2) Extract needs and trade-offs tidy tables ----
def extract_needs(df, device_type):
    out = df.copy()
    out["device_type"] = device_type
    # Boolean feasibility flags based on thresholds
    out["need_ram_ok"] = out["ram_gb"].ge(NEEDS_THRESHOLDS["ram_gb"]) if "ram_gb" in out else True
    out["need_storage_ok"] = out["storage_tb"].ge(NEEDS_THRESHOLDS["storage_tb"]) if "storage_tb" in out else True
    out["need_battery_ok"] = out["battery_hours"].ge(NEEDS_THRESHOLDS["battery_hours"]) if "battery_hours" in out else True
    out["need_weight_ok"] = out["weight_kg"].le(NEEDS_THRESHOLDS["weight_kg"]) if "weight_kg" in out else True
    return out[["name","device_type"] + [c for c in NEEDS_COLS if c in out.columns] +
               ["need_ram_ok","need_storage_ok","need_battery_ok","need_weight_ok"]]

def extract_tradeoffs(df, device_type):
    out = df.copy()
    out["device_type"] = device_type
    return out[["name","device_type"] + [c for c in TRADEOFF_COLS if c in out.columns]]

needs_mac = extract_needs(mac_df, "mac")
needs_ipad = extract_needs(ipad_df, "ipad")
trade_mac = extract_tradeoffs(mac_df, "mac")
trade_ipad = extract_tradeoffs(ipad_df, "ipad")

needs_tbl = pd.concat([needs_mac, needs_ipad], ignore_index=True)
trade_tbl = pd.concat([trade_mac, trade_ipad], ignore_index=True)

# ---- 3) Build simple bundles (Mac only; Mac + iPad) to show how needs combine ----
def make_bundles(mac_df, ipad_df):
    bundles = []

    # Mac-only bundles
    for _, m in mac_df.iterrows():
        bundles.append({
            "bundle": f"{m['name']} (Mac only)",
            "devices": json.dumps([m["name"]]),
            "price_inr": m["price_inr"],
            "carry_weight_kg": m["weight_kg"],
            "battery_hours_min": m["battery_hours"],
            "ram_gb_sum": m["ram_gb"],
            "storage_tb_sum": m["storage_tb"],
            "learning_hours_sum": (m.get("learning_hours") or 1.0),
            "maintenance_hours_per_year_sum": (m.get("maintenance_hours_per_year") or 2.0),
            "power_adequacy_score_avg": (m.get("power_adequacy_score") or 0.0),
        })

    # Mac + iPad bundles (sum/aggregate where appropriate)
    for _, m in mac_df.iterrows():
        for _, i in ipad_df.iterrows():
            bundles.append({
                "bundle": f"{m['name']} + {i['name']}",
                "devices": json.dumps([m["name"], i["name"]]),
                "price_inr": (m["price_inr"] + i["price_inr"]),
                "carry_weight_kg": (m["weight_kg"] + i["weight_kg"]),
                "battery_hours_min": min(m["battery_hours"], i["battery_hours"]),
                "ram_gb_sum": m["ram_gb"],              # iPad RAM not provided; keep Mac RAM as binding
                "storage_tb_sum": (m["storage_tb"] + (i["storage_tb"] or 0)),
                "learning_hours_sum": (m.get("learning_hours") or 1.0) + (i.get("learning_hours") or 0.8),
                "maintenance_hours_per_year_sum": (m.get("maintenance_hours_per_year") or 2.0) + (i.get("maintenance_hours_per_year") or 1.0),
                "power_adequacy_score_avg": pd.Series([m.get("power_adequacy_score"), i.get("power_adequacy_score")]).astype(float).mean(skipna=True),
            })
    return pd.DataFrame(bundles)

bundles_tbl = make_bundles(mac_df, ipad_df)

bundles_tbl

,bundle,devices,price_inr,carry_weight_kg,battery_hours_min,ram_gb_sum,storage_tb_sum,learning_hours_sum,maintenance_hours_per_year_sum,power_adequacy_score_avg
0,MacBook Air 13″ (M4) (Mac only),"[""MacBook Air 13\u2033 (M4)""]",99900.0,1.24,15.0,16,2.000,1.0,4.0,7.5
1,MacBook Air 15″ (M4) (Mac only),"[""MacBook Air 15\u2033 (M4)""]",124900.0,1.51,15.0,16,2.000,1.0,4.0,7.5
2,MacBook Pro 14″ (M4) (Mac only),"[""MacBook Pro 14\u2033 (M4)""]",169900.0,1.55,16.0,16,2.000,1.5,5.0,8.3
3,MacBook Pro 14″ (M4 Pro or M4 Max) (Mac only),"[""MacBook Pro 14\u2033 (M4 Pro or M4 Max)""]",199900.0,1.60,14.0,24,8.000,2.0,6.0,9.2
4,MacBook Pro 16″ (M4 Pro or M4 Max) (Mac only),"[""MacBook Pro 16\u2033 (M4 Pro or M4 Max)""]",249900.0,2.14,17.0,24,8.000,2.0,6.0,9.6
...,...,...,...,...,...,...,...,...,...,...
65,Mac Pro (M2 Ultra) + iPad Pro 13″ (M4),"[""Mac Pro (M2 Ultra)"", ""iPad Pro 13\u2033 (M4)""]",859800.0,NaN,NaN,64,10.000,NaN,NaN,9.2
66,Mac Pro (M2 Ultra) + iPad Air 11″ (M3),"[""Mac Pro (M2 Ultra)"", ""iPad Air 11\u2033 (M3)""]",789800.0,NaN,NaN,64,9.000,NaN,NaN,9.2
67,Mac Pro (M2 Ultra) + iPad Air 13″ (M3),"[""Mac Pro (M2 Ultra)"", ""iPad Air 13\u2033 (M3)""]",809800.0,NaN,NaN,64,9.000,NaN,NaN,9.2
68,Mac Pro (M2 Ultra) + iPad 11″ (A16),"[""Mac Pro (M2 Ultra)"", ""iPad 11\u2033 (A16)""]",764800.0,NaN,NaN,64,8.512,NaN,NaN,9.2


In [6]:


# Add deterministic "needs" feasibility to bundles
bundles_tbl["need_ram_ok"] = bundles_tbl["ram_gb_sum"] >= NEEDS_THRESHOLDS["ram_gb"]
bundles_tbl["need_storage_ok"] = bundles_tbl["storage_tb_sum"] >= NEEDS_THRESHOLDS["storage_tb"]
bundles_tbl["need_battery_ok"] = bundles_tbl["battery_hours_min"] >= NEEDS_THRESHOLDS["battery_hours"]
bundles_tbl["need_weight_ok"] = bundles_tbl["carry_weight_kg"] <= NEEDS_THRESHOLDS["weight_kg"]
bundles_tbl["feasible_all_needs"] = bundles_tbl[
    ["need_ram_ok","need_storage_ok","need_battery_ok","need_weight_ok"]
].all(axis=1)

# Save outputs
needs_path = f"{PARENT_DIR}/extracted_needs_constraints.csv"
trade_path = f"{PARENT_DIR}/extracted_tradeoffs.csv"
bundles_path = f"{PARENT_DIR}/bundles_with_needs_and_tradeoffs.csv"
needs_tbl.to_csv(needs_path, index=False)
trade_tbl.to_csv(trade_path, index=False)
bundles_tbl.to_csv(bundles_path, index=False)

# Display tables
#print(needs_tbl)
#display_dataframe_to_user("Real-world trade-offs extracted", trade_tbl)
#display_dataframe_to_user("Bundles with needs feasibility + trade-offs", bundles_tbl)

print(f"Saved:\n- Needs: {needs_path}\n- Trade-offs: {trade_path}\n- Bundles: {bundles_path}")


Saved:
- Needs: apple_data/extracted_needs_constraints.csv
- Trade-offs: apple_data/extracted_tradeoffs.csv
- Bundles: apple_data/bundles_with_needs_and_tradeoffs.csv
